# scrapli

In [ ]:
# Basic connection
from scrapli import Scrapli

device = {
   "host": "10.10.20.48",
   "auth_username": "developer",
   "auth_password": "C1sco12345",
   "auth_strict_key": False,
   "platform": "cisco_iosxe"
}

conn = Scrapli(**device)
conn.open()
print(conn.get_prompt())

In [ ]:
conn.close()

In [ ]:
# Using context manager
with Scrapli(**device) as conn:
    print(conn.get_prompt())

In [ ]:
print(conn.send_command("show version"))

In [ ]:
# Seeing the commands sent
sh_ver = conn.send_command("show version")
print(sh_ver.result)

In [ ]:
conn.send_commands(["show run", "show version"])

In [ ]:
conn.send_commands_from_file("commands.txt")

In [ ]:
# Using driver
from scrapli.driver.core import IOSXEDriver

with IOSXEDriver(**device) as conn:
    result = conn.send_command("show run")
    print(result.result)

In [ ]:
# Use GenericDriver for non-core platforms
# Use Driver as the last resort

In [ ]:
# Parsing with Genie
from pprint import pprint
response = conn.send_command("show version")
pprint(response.genie_parse_output())

In [ ]:
# Parsing with TextFSM
pprint(response.textfsm_parse_output())

In [ ]:
# ssh config file 
from scrapli.driver.core import IOSXEDriver

my_device = {
    "host": "10.10.20.48",
    "auth_username": "developer",
    "auth_password": "C1sco12345",
    "auth_strict_key": False,
    "ssh_config_file": "~/.ssh/config",
}

with IOSXEDriver(**my_device) as conn:
    print(conn.get_prompt())

In [ ]:
# Sending configuration commands
with Scrapli(**device) as conn:
   config_commands = [
        "interface Loopback0",
        "description Created with Scrapli",
        "ip address 1.1.1.1 255.255.255.255",
    ]

   result = conn.send_configs(config_commands)

print(result.result)
print("Interface Loopback0 created successfully!")

In [ ]:
# Privilege levels
with IOSXEDriver(**my_device) as conn:
    conn.acquire_priv("exec")
    # conn.acquire_priv("privilege_exec")
    # conn.acquire_priv("configuration")

In [ ]:
# Use send_config for single command
conn.send_config("int loopback 100")

In [ ]:
# Handling prompts
with IOSXEDriver(**device) as conn:
    interactive = conn.send_interactive(
            [("copy run start", "Destination filename [startup-config]?", "\n")],
    )
    print(interactive.result)

In [ ]:
# The 'On Open' feature
from scrapli.driver.core import IOSXEDriver

def config_mode(conn):
    response = conn.channel.send_input("conf t")

my_device = {
    "host": "10.10.20.48",
    "auth_username": "developer",
    "auth_password": "C1sco12345",
    "auth_strict_key": False,
    "on_open": config_mode
}

with IOSXEDriver(**my_device) as conn:
    print(conn.get_prompt())

In [ ]:
# The 'On Close' feature
from scrapli.driver.core import IOSXEDriver

def show_int(conn):
    response = conn.channel.send_input("sh ip int br")

my_device = {
    "host": "10.10.20.48",
    "auth_username": "developer",
    "auth_password": "C1sco12345",
    "auth_strict_key": False,
    "on_close": show_int
}

with IOSXEDriver(**my_device) as conn:
    print(conn.get_prompt())

In [ ]:
# Logging
import logging
from scrapli.driver.core import IOSXEDriver

logging.basicConfig(filename="scrapli.log", level=logging.DEBUG)

device = {
    "host": "10.10.20.47",
    "auth_username": "developer",
    "auth_password": "C1sco12345",
    "auth_strict_key": False,
}

conn = IOSXEDriver(**device)
conn.open()
print(conn.get_prompt())
print(conn.send_command("show run | i hostname").result)


In [ ]:
# Logging 2
from scrapli import Scrapli
from scrapli.logging import enable_basic_logging

# the `enable_basic_logging` function accepts a bool or a string for the `file` argument -- if you
# provide a string that string will be used as the output path for the log file, if you just pass
# `True` as in this example, a file called "scrapli.log" will be created in your working directory
enable_basic_logging(file=True, level="debug")

device = {
   "host": "10.10.20.48",
   "auth_username": "developer",
   "auth_password": "C1sco12345",
   "auth_strict_key": False,
   "platform": "cisco_iosxe"
}

with Scrapli(**device) as conn:
    print(conn.get_prompt())
    print(conn.send_command("show run | i hostname").result)

In [ ]:
# motd banner
from scrapli.driver.core import IOSXEDriver

device = {
   "host": "10.10.20.48",
   "auth_username": "developer",
   "auth_password": "C1sco12345",
   "auth_strict_key": False,
}


conn = IOSXEDriver(**device)
conn.open()

my_banner = """banner motd ^
Configured via Scrapli!
^
"""
# Because the banner "input mode" is basically like a text editor where we dont get the prompt
# printed out between sending lines of banner config we need to use the `eager` mode to force
# scrapli to blindly send the banner/macro lines without looking for the prompt in between each
# line. You should *not* use eager unless you need to and know what you are doing as it
# basically disables one of the core features that makes scrapli reliable!
result = conn.send_config(config=my_banner, eager=True)
print(result.result)

In [ ]:
# Async programming
import asyncio

async def say_hello():
    print("Hello")
    await asyncio.sleep(1)  # Simulate a delay of 1 second
    print("World!")

async def main():
    task1 = asyncio.create_task(say_hello())
    task2 = asyncio.create_task(say_hello())

    await task1
    await task2

asyncio.run(main())

In [ ]:
# Syncronous connections
from scrapli.driver.core import IOSXEDriver, NXOSDriver
from time import time

start = time()

IOSXE_DEVICE = {
    "host": "10.10.20.48",
    "auth_username": "developer",
    "auth_password": "C1sco12345",
    "auth_strict_key": False,
}

NXOS_DEVICE = {
    "host": "10.10.20.40",
    "auth_username": "admin",
    "auth_password": "RG!_Yw200",
    "auth_strict_key": False,
}

with IOSXEDriver(**IOSXE_DEVICE) as conn:
   print(conn.get_prompt())

with NXOSDriver(**NXOS_DEVICE) as conn:
    print(conn.get_prompt())

end = time()

print(f"Finished in {end - start:.2f}s")

In [ ]:
# Asyncronous connections
import asyncio
from scrapli.driver.core import AsyncIOSXEDriver, AsyncNXOSDriver
from time import time

start = time()

DEVICES = [
    {
        "host": "10.10.20.48",
        "auth_username": "developer",
        "auth_password": "C1sco12345",
        "auth_strict_key": False,
        "transport": "asyncssh",
        "driver": AsyncIOSXEDriver,
    },

    {
        "host": "10.10.20.40",
        "auth_username": "admin",
        "auth_password": "RG!_Yw200",
        "auth_strict_key": False,
        "transport": "asyncssh",
        "driver": AsyncNXOSDriver,
    }
]


async def gather_version(device):
    driver = device.pop("driver")
    conn = driver(**device)
    await conn.open()
    prompt_result = await conn.get_prompt()
    await conn.close()
    return prompt_result


async def main():
    coroutines = [gather_version(device) for device in DEVICES]
    results = await asyncio.gather(*coroutines)
    print(results)


asyncio.get_event_loop().run_until_complete(main())

end = time()

print(f"Finished in {end - start:.2f}s")

# scrapli_netconf

In [ ]:
from scrapli_netconf.driver import NetconfDriver

my_device = {
    "host": "10.10.20.48",
    "auth_username": "developer",
    "auth_password": "C1sco12345",
    "auth_strict_key": False,
    "port": 830
}

conn = NetconfDriver(**my_device)
conn.open()
response = conn.get_config(source="running")
print(response.result)

# scrapli_cfg

In [ ]:
from scrapli import Scrapli
from scrapli_cfg import ScrapliCfg

device = {
   "host": "10.10.20.48",
   "auth_username": "developer",
   "auth_password": "C1sco12345",
   "auth_strict_key": False,
   "platform": "cisco_iosxe"
}

with open("myconfig", "r") as f:
    my_config = f.read()

with Scrapli(**device) as conn:
  cfg_conn = ScrapliCfg(conn=conn)
  cfg_conn.prepare()
  cfg_conn.load_config(config=my_config, replace=True)
  diff = cfg_conn.diff_config()
  print(diff.side_by_side_diff)
  cfg_conn.commit_config()
  cfg_conn.cleanup()